In [4]:
import os
import shutil
import pandas as pd
import oracledb

DB_USER = "plocharczykd"
DB_PASSWORD = "dawid"
DB_DSN = "213.184.8.44:1521/orcl"
DB_HOST = "213.184.8.44"
DB_PORT = "1521"
DB_SID = "orcl"
DSN = oracledb.makedsn(DB_HOST, DB_PORT, DB_SID)
CSV_DIR = "dane"
ARCHIVE_DIR = "archiwum"



# Funkcja sprawdzająca, czy dane w CSV są zgodne z wymaganiami
def sprawdz_dane(df, tabela):
    if tabela == "firmy":
        wymagane_kolumny = ["firma_id", "nazwa", "adres", "telefon", "email"]
    elif tabela == "stanowiska":
        wymagane_kolumny = ["stanowisko_id", "firma_id", "nazwa", "opis", "wymagania", "wynagrodzenie", "data_dodania"]
    elif tabela == "kandydaci":
        wymagane_kolumny = ["kandydat_id", "imie", "nazwisko", "email", "telefon", "pesel", "data_aplikacji"]
    elif tabela == "aplikacje":
        wymagane_kolumny = ["aplikacja_id", "kandydat_id", "stanowisko_id", "data_aplikacji", "status"]
    elif tabela == "logi":
        wymagane_kolumny = ["id_logu", "akcja", "szczegoly", "data_operacji"]
    elif tabela == "podsumowanie":
        wymagane_kolumny = ["id_podsumowania", "typ_podsumowania", "okres", "liczba_aplikacji", "data_wygenerowania"]
    else:
        raise ValueError(f"Nieznana tabela: {tabela}")

    for kolumna in wymagane_kolumny:
        if kolumna not in df.columns:
            raise ValueError(f"Brakuje kolumny: {kolumna}")
    return True

# Funkcja ładująca dane z csv
def laduj_dane(csv_file, tabela):
    try:
        df = pd.read_csv(csv_file)
        df.columns = df.columns.str.strip()

        # Dostosowanie dat
        if 'data_aplikacji' in df.columns:
            df['data_aplikacji'] = pd.to_datetime(df['data_aplikacji'])
        if 'data_dodania' in df.columns:
            df['data_dodania'] = pd.to_datetime(df['data_dodania'])

        sprawdz_dane(df, tabela)

        # Połączenie z bazą danych Oracle
        conn = oracledb.connect(user=DB_USER, password=DB_PASSWORD, dsn=DSN)
        cursor = conn.cursor()

        # Przesyłanie danych do bazy
        for index, row in df.iterrows():
            if tabela == "aplikacje":
                columns = df.columns[df.columns != 'aplikacja_id']  # lub inna kolumna tożsamości
            elif tabela == "kandydaci":
                columns = df.columns[df.columns != 'kandydat_id']
            else:
                columns = df.columns  # Dla innych tabel, jeżeli nie ma kolumn tożsamości

            placeholders = ', '.join([f":{i}" for i in range(1, len(columns) + 1)])
            values = [row[col] if not pd.isna(row[col]) else None for col in columns]

            sql = f"INSERT INTO {tabela} ({', '.join(columns)}) VALUES ({placeholders})"

            cursor.execute(sql, {f":{i}": val for i, val in enumerate(values, start=1)})

        conn.commit()
        cursor.close()
        conn.close()

        print(f"Pomyślnie załadowano dane z {csv_file} do tabeli {tabela}.")
    except Exception as e:
        print(f"Błąd podczas ładowania danych z {csv_file}: {e}")

# Funkcja archiwizująca csv
def archiwizuj_plik(csv_file):
    if not os.path.exists(ARCHIVE_DIR):
        os.makedirs(ARCHIVE_DIR)
    shutil.move(csv_file, os.path.join(ARCHIVE_DIR, os.path.basename(csv_file)))
    print(f"Zarchiwizowano plik: {csv_file}")

def main():
    for file in os.listdir(CSV_DIR):
        if file.endswith(".csv"):
            csv_file = os.path.join(CSV_DIR, file)

            if "firmy" in file.lower():
                tabela = "firmy"
            elif "stanowiska" in file.lower():
                tabela = "stanowiska"
            elif "kandydaci" in file.lower():
                tabela = "kandydaci"
            elif "aplikacje" in file.lower():
                tabela = "aplikacje"
            elif "logi" in file.lower():
                tabela = "logi"
            elif "podsumowanie" in file.lower():
                tabela = "podsumowanie"
            else:
                print(f"Nieznany typ pliku: {file}")
                continue

            laduj_dane(csv_file, tabela)
            archiwizuj_plik(csv_file)

if __name__ == "__main__":
    main()


Błąd podczas ładowania danych z dane\kandydaci.csv: ORA-00001: naruszono więzy unikatowe (PLOCHARCZYKD.SYS_C00229251)
Help: https://docs.oracle.com/error-help/db/ora-00001/
Zarchiwizowano plik: dane\kandydaci.csv
